In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

def load_ct_image(image_path):
    """Load a CT scan image from the specified path."""
    ct_image = Image.open('/content/drive/MyDrive/ID_0062_AGE_0067_CONTRAST_0_CT.jpg')
    ct_image_cv = cv2.cvtColor(np.array(ct_image), cv2.COLOR_RGB2BGR)
    return ct_image, ct_image_cv

def save_manipulated_image(manipulated_image, output_path):
    """Save the manipulated CT scan image to the specified path."""
    manipulated_ct = Image.fromarray(cv2.cvtColor(manipulated_image, cv2.COLOR_BGR2RGB))
    manipulated_ct.save(output_path)
    print(f"Manipulated CT image saved to: {output_path}")

def apply_gaussian_blur(ct_image_cv, roi_x, roi_y, roi_width, roi_height, kernel_size=(5, 5)):
    """Apply Gaussian blur to the specified ROI of the CT scan image."""
    roi = ct_image_cv[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
    blurred_roi = cv2.GaussianBlur(roi, kernel_size, 0)
    ct_image_cv[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width] = blurred_roi
    return ct_image_cv

def apply_contrast_adjustment(ct_image_cv, roi_x, roi_y, roi_width, roi_height, alpha=1.2, beta=20):
    """Adjust the contrast of the specified ROI of the CT scan image."""
    roi = ct_image_cv[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
    adjusted_roi = cv2.convertScaleAbs(roi, alpha=alpha, beta=beta)
    ct_image_cv[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width] = adjusted_roi
    return ct_image_cv

def apply_edge_enhancement(ct_image_cv, roi_x, roi_y, roi_width, roi_height, kernel_size=(3, 3)):
    """Apply edge enhancement to the specified ROI of the CT scan image."""
    roi = ct_image_cv[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
    edge_kernel = np.array([[-1, -1, -1],
                           [-1, 8, -1],
                           [-1, -1, -1]])
    enhanced_roi = cv2.filter2D(roi, -1, edge_kernel)
    ct_image_cv[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width] = enhanced_roi
    return ct_image_cv

def main():
    # Load the target CT scan image
    target_ct_path = '/content/drive/MyDrive/ID_0061_AGE_0074_CONTRAST_0_CT.jpg'
    target_ct, target_ct_cv = load_ct_image(target_ct_path)

    # Define the region of interest (ROI) for manipulation
    roi_x, roi_y, roi_width, roi_height = 100, 200, 300, 400

    # Apply Gaussian blur to the ROI
    manipulated_ct_cv = apply_gaussian_blur(target_ct_cv, roi_x, roi_y, roi_width, roi_height)

    # Apply contrast adjustment to the ROI
    manipulated_ct_cv = apply_contrast_adjustment(manipulated_ct_cv, roi_x, roi_y, roi_width, roi_height)

    # Apply edge enhancement to the ROI
    manipulated_ct_cv = apply_edge_enhancement(manipulated_ct_cv, roi_x, roi_y, roi_width, roi_height)

    # Save the manipulated CT scan image
    output_path = "/content/drive/MyDrive/manipulated_ct_2.png"
    save_manipulated_image(manipulated_ct_cv, output_path)

if __name__ == "__main__":
    main()

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, f1_score
import tifffile
from scipy.spatial.distance import cosine
import skimage.feature
from skimage.feature import greycoprops, graycomatrix

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

def load_ct_image(image_path):
    """Load a CT scan image from the specified path."""
    try:
        if image_path.endswith('.tif') or image_path.endswith('.tiff'):
            ct_image = tifffile.imread(image_path)
            if ct_image.dtype == np.float64:
                ct_image = ct_image.astype(np.float32)
            ct_image = cv2.resize(ct_image, (224, 224))
            ct_image = cv2.cvtColor(ct_image, cv2.COLOR_GRAY2RGB)
        else:
            ct_image = cv2.imread(image_path)
            ct_image = cv2.resize(ct_image, (224, 224))
        return ct_image
    except Exception as e:
        print(f"Error loading image: {image_path} - {e}")
        return None

def extract_features(image, model):
    """Extract features from the CT scan image using the pre-trained model."""
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    features = model.predict(image)
    return features.flatten()

def extract_combined_features(image, model):
    """Extract a combination of features from the CT scan image."""
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    vgg16_features = model.predict(image).flatten()

    # Extract histogram features
    hist, _ = np.histogram(image.squeeze(), bins=32, range=(0, 255))
    hist_features = hist / np.sum(hist)

    # Extract texture features
    # Convert the image to grayscale before computing GLCM
    gray_image = cv2.cvtColor(image.squeeze(), cv2.COLOR_RGB2GRAY)
    glcm = graycomatrix(gray_image.astype(np.uint8), distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)
    contrast = skimage.feature.graycoprops(glcm, 'contrast')[0, 0]  # Updated
    dissimilarity = skimage.feature.graycoprops(glcm, 'dissimilarity')[0, 0]  # Updated
    homogeneity = skimage.feature.graycoprops(glcm, 'homogeneity')[0, 0]  # Updated
    energy = skimage.feature.graycoprops(glcm, 'energy')[0, 0]  # Updated
    correlation = skimage.feature.graycoprops(glcm, 'correlation')[0, 0]  # Updated
    texture_features = [contrast, dissimilarity, homogeneity, energy, correlation]

    return np.concatenate([vgg16_features, hist_features, texture_features])

def find_most_similar_original(manipulated_image, original_paths):
    """Find the original image that is most similar to the manipulated image."""
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    manipulated_feature = extract_combined_features(manipulated_image, model)

    min_distance = float('inf')
    most_similar_path = None

    for path in original_paths:
        original_image = load_ct_image(path)
        original_feature = extract_combined_features(original_image, model)
        distance = np.linalg.norm(manipulated_feature - original_feature)
        if distance < min_distance:
            min_distance = distance
            most_similar_path = path

    return most_similar_path

# Test the function with the provided potential_original_paths
potential_original_paths = [

    "/content/drive/MyDrive/ID_0000_AGE_0060_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0001_AGE_0069_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0002_AGE_0074_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0003_AGE_0075_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0004_AGE_0056_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0005_AGE_0048_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0006_AGE_0075_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0007_AGE_0061_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0008_AGE_0051_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0009_AGE_0048_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0010_AGE_0060_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0011_AGE_0061_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0012_AGE_0061_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0013_AGE_0060_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0014_AGE_0071_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0015_AGE_0061_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0016_AGE_0063_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0017_AGE_0060_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0018_AGE_0074_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0019_AGE_0070_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0020_AGE_0066_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0021_AGE_0067_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0022_AGE_0074_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0023_AGE_0061_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0024_AGE_0060_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0025_AGE_0074_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0026_AGE_0070_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0027_AGE_0064_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0028_AGE_0074_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0029_AGE_0078_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0030_AGE_0076_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0031_AGE_0039_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0032_AGE_0061_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0033_AGE_0071_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0034_AGE_0061_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0035_AGE_0059_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0036_AGE_0074_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0037_AGE_0074_CONTRAST_1_CT.tif",
    "/content/drive/MyDrive/ID_0062_AGE_0067_CONTRAST_0_CT.tif"
]

# Assuming you have a manipulated image, e.g., /content/drive/MyDrive/manipulated_image.tif
manipulated_image = load_ct_image("/content/drive/MyDrive/manipulated_ct_2.tiff")
most_similar_original = find_most_similar_original(manipulated_image, potential_original_paths)
print(f"The most similar original image is: {most_similar_original}") 